### Libraries

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [97]:
tf.__version__

'2.9.1'

## `1) Data Preprocessing`

### Importing Dataset

In [98]:
dataset = pd.read_csv("Churn_Modelling.csv")

In [99]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [100]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Splitting Dataset into X and y

In [101]:
# X = dataset.iloc[:, 3:-1].values # iloc[:, 3:-1] means all rows and columns from 3 to -1
# y = dataset.iloc[:, -1].values # iloc[:, -1] means all the rows and only the last column
# OR
X = dataset.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1).values
y = dataset['Exited'].values

In [102]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [103]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding Categorical Data

#### Label Encoding "Gender"

In [104]:
from sklearn.preprocessing import LabelEncoder # LabelEncoder is used to encode categorical data
le = LabelEncoder() 
X[:, 2] = le.fit_transform(X[:, 2])
# OR get_dummies before converting X to X.values

In [105]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding "Geography" Column

In [106]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder # OneHotEncoder is used to encode categorical data
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])],
                       remainder="passthrough") # passthrough means that the other columns will not be transformed
X = np.array(ct.fit_transform(X))

In [107]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting Dataset into Training and Test Set

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)

### Feature Scaling
Important Note: Feature Scaling is a **must** for Deep Learning.  
It is not mandatory for ANN, but it is a **good practice** to do it.  
It will help us to **converge faster** to the optimal weights.

In [109]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## `2) Building ANN`

### Initializing ANN

In [110]:
ann = tf.keras.models.Sequential() # Sequential is used to initialize the ANN

### Adding Input Layer and First Hidden Layer  
Dense is used to add the input layer and the first hidden layer

In [111]:
ann.add(tf.keras.layers.Dense(units=6,
                              activation="relu")) # relu: rectified linear activation unit

### Adding Second Hidden Layer

In [112]:
ann.add(tf.keras.layers.Dense(units=6,
                              activation="relu"))

### Adding Output Layer  
**Single Neuron** and **sigmoid** activation function in the output layer  
because we have a **binary outcome**.

In [113]:
ann.add(tf.keras.layers.Dense(units=1,
                              activation="sigmoid")) # 0 or 1 (Exited or not)

## `3) Training ANN`

### Compiling ANN

In [114]:
ann.compile(optimizer="adam", # adam is a stochastic gradient descent algorithm
            loss="binary_crossentropy", # binary_crossentropy is used for binary classification
            metrics=["accuracy"]) # accuracy is used to evaluate the model

### Training ANN on Training Set

In [115]:
ann.fit(X_train, y_train,
        batch_size=32, # batch_size: no. of observations weights are updated
        epochs=100) # epochs: no. of times the whole training set is passed through the ANN

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.7191 - accuracy: 0.5691
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5324 - accuracy: 0.7947
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4728 - accuracy: 0.8016
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4384 - accuracy: 0.8145
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4196 - accuracy: 0.8227
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4038 - accuracy: 0.8322
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3874 - accuracy: 0.8403
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.8494
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3651 - accuracy: 0.8549
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3586 - accura

## `4) Predictions and Evaluation`

### Predicting for Test Set
predict() returns the probability of the person leaving the bank

In [116]:
predictions = ann.predict(X_test)
predictions = (predictions > 0.5)
print(np.concatenate((predictions.reshape(len(predictions), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Evaluating the Model

In [117]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[1513   82]
 [ 196  209]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1595
           1       0.72      0.52      0.60       405

    accuracy                           0.86      2000
   macro avg       0.80      0.73      0.76      2000
weighted avg       0.85      0.86      0.85      2000

0.861


### Homework

In [118]:
person = [1,0,0,600,1,40,3,60000,2,1,1,50000]
print(ann.predict(sc.transform([person])) > 0.5)

1/1 [==============================] - 0s 18ms/step
[[False]]
